# Библиотеки Python для DS (семинары)

Урок 10. Практическое применение метода K-Means. Иерархическая кластеризация

Цель: Проанализировать данные о заработной плате сотрудников из файла ds_salaries.csv и построить модели машинного обучения для прогнозирования заработной платы.

Задачи:

1. Первичный анализ данных:
— Импортировать данные из файла ds_salaries.csv.
— Очистить данные от дубликатов и пропущенных значений.
— Исследовать типы данных, описательную статистику и корреляции между признаками.
— Визуализировать данные с помощью Matplotlib и Seaborn.

2. Построение модели линейной регрессии:
— Разделить данные на обучающую и тестовую выборки.
— Построить модель линейной регрессии для прогнозирования заработной платы.
— Оценить качество модели с помощью метрик MAE, MSE и R-квадрат.
— Проанализировать значимость признаков и мультиколлинеарность.

3. Построение модели дерева решений:
— Построить модель дерева решений для прогнозирования заработной платы.
— Оценить качество модели с помощью метрик RMSE и R-квадрат.
— Подберите гиперпараметры для модели дерево решений
— Визуализировать дерево решений.

4. Построение ансамблевой модели:
— Построить ансамблевую модель, используя Random Forest и Boosting.
— Сравнить качество ансамблевой модели с отдельными моделями.
— Оценить важность признаков в ансамблевой модели.

5. Понижение размерности:
— Применить Principal Component Analysis (PCA) для снижения размерности данных.
— Построить модель линейной регрессии с использованием данных, полученных после PCA.
— Сравнить качество модели с использованием исходных данных.

7. Кластеризация:
— Применить метод K-Means для кластеризации данных по заработной плате.
— Определить оптимальное количество кластеров.
— Проанализировать характеристики полученных кластеров.
— Визуализировать результаты кластеризации и проанализировать характеристики в каждом кластере.
— Проанализируйте полученные данные и сделайте выводы о том, какой алгоритм кластеризации сможет выделить кластеры лучше, чем K-Means.

## Выполнение домашней работы

##### Используем предоставленый к заданию файл fetch_california_housing.xlsx

In [3]:
# Библиотеки для работы с датасетом и графиками
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_distances
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score

##### 1. Первичный анализ данных:

— Импортировать данные из файла ds_salaries.csv.

— Очистить данные от дубликатов и пропущенных значений.

— Исследовать типы данных, описательную статистику и корреляции между признаками.

— Визуализировать данные с помощью Matplotlib и Seaborn.

In [4]:
# Загружаем датасет
df = pd.read_csv('ds_salaries.csv')

In [5]:
# Выводим дата фрейм
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


work_year: Это конкретный год, в котором была выплачена заработная плата. В разные годы могут быть разные экономические условия, которые могут повлиять на уровень заработной платы.

experience_level: уровень опыта, который человек имеет на конкретной работе. Это является ключевым фактором при расчете заработной платы, поскольку, как правило, более опытные сотрудники получают более высокую зарплату благодаря своим передовым навыкам и знаниям.

employment_type: Характер трудового договора, например, полный рабочий день, неполный рабочий день или по контракту, может существенно влиять на заработную плату. Сотрудники, работающие полный рабочий день, часто получают более высокую годовую заработную плату по сравнению со своими коллегами, работающими неполный рабочий день или по контракту.

job_title: Должность, которую человек занимает в компании. Разные должности имеют разную шкалу заработной платы в зависимости от требуемых обязанностей и навыков. Например, на руководящих должностях обычно платят больше, чем на должностях начального уровня.

salary: общая сумма заработной платы, выплачиваемой сотруднику. На это напрямую влияют такие факторы, как уровень опыта, название должности и тип занятости.

salary_currency: Конкретная валюта, в которой выплачивается заработная плата, обозначаемая кодом ISO 4217. Обменные курсы могут влиять на величину заработной платы при пересчете в разные валюты.

salary_in_usd: Общая сумма заработной платы брутто, пересчитанная в доллары США. Это позволяет проводить единообразное сравнение заработной платы в разных странах и валютах.

employee_residence: Основная страна проживания сотрудника, обозначаемая кодом ISO 3166. Стоимость жизни и преобладающие ставки заработной платы в стране проживания сотрудника могут влиять на уровень заработной платы.

remote_ratio: доля работы, выполняемой удаленно. С ростом удаленной работы компании могут корректировать заработную плату в зависимости от стоимости жизни в месте нахождения сотрудника и доли удаленной работы.

company_location: местоположение главного офиса работодателя или филиала, с которым заключен контракт. Компании в разных регионах могут предлагать разные уровни заработной платы из-за различных экономических условий и стоимости жизни.

company_size:  Среднее число сотрудников в компании в течение рабочего года. Крупные компании часто имеют структурированную шкалу заработной платы и могут предлагать более высокие оклады за счет эффекта масштаба и увеличения доходов.

In [6]:
# Выводим общую информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   int64 
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(4), object(7)
memory usage: 322.8+ KB


Из общей информации следует:
1. Всего 10 колонок, DataFrame содержит 3755 записи, от 0 до 3754
2. Нулевые значения отсуствуют
3. Пропущеные значения отсуствуют
4. Колонки: work_year, salary, salary_in_usd, remote_ratio находятся в целочисленых значениях int64
5. Колонки: experience_level, employment_type, job_title, salary_currency, employee_residence, company_location, company_size являются объектами object

In [7]:
# Статистический анализ числовых переменных
df.describe().T

,count,mean,std,min,25%,50%,75%,max
work_year,3755.0,2022.373635,0.691448,2020.0,2022.0,2022.0,2023.0,2023.0
salary,3755.0,190695.571771,671676.500508,6000.0,100000.0,138000.0,180000.0,30400000.0
salary_in_usd,3755.0,137570.389880,63055.625278,5132.0,95000.0,135000.0,175000.0,450000.0
remote_ratio,3755.0,46.271638,48.589050,0.0,0.0,0.0,100.0,100.0


На основе представленных статистических данных неприрывных признаков можно сделать следующие выводы:

1. work_year (Год выплаты зарплаты): 
- Средний уровень выплаты зарплаты на уровне 2022
- Миниманая 2020
- Максимальная 2023
- Медиана равна 190695,58
- Отклонение состовляет 671676,50
2. salary (Сумма зарплаты):
- Средняя сумма зарплаты на уровне 138000.00
- Миниманая 6000.00
- Максимальная 30400000.00
- Медиана равна 190695.58
- Отклонение состовляет 671676.50
3. salary_in_usd (Сумма зарплаты в USD):
- Средняя сумма зарплаты в USD на уровне 135000.00
- Миниманая 5132.00
- Максимальная 450000.00
- Медиана равна 137570.39
- Отклонение состовляет 63055.63
4. remote_ratio (доля работы выполняемой удалённо):
- Средние доля работы выполняемой удалёно на уровне 0.00
- Миниманая 0.00
- Максимальная 100.00
- Медиана равна 46.27
- Отклонение состовляет 48.59

In [8]:
# Статистический анализ объектных признаков
df.describe(include='object').T

,count,unique,top,freq
experience_level,3755,4,SE,2516
employment_type,3755,4,FT,3718
job_title,3755,93,Data Engineer,1040
salary_currency,3755,20,USD,3224
employee_residence,3755,78,US,3004
company_location,3755,72,US,3040
company_size,3755,3,M,3153


1. experience_level (Опыт работы):
- В данных присутствуют записи о поле у 3755 записей.
- Имеются четыре уникальные записи о опыте работы.
- Наиболее часто встречающийся запись - SE, с частотой 2516 раз.

2. employment_type (Характер трудового договора):
- Для 3755 записей о трудовых договорах.
- Имеются четыре уникальные записи трудового договора.
- Наиболее часто встречающийся запись - FT, с частотой 3718 раз.

3. job_title (Должность):
- Информация о должностях состовляет 3755 записей.
- Имеется 93 уникальные должности.
- Наиболее распространенная должность Data Engineer, встречается 1040 раз.

4. salary_currency (Вид валюты):
- Информация о видах валюты состовляет 3755 записей.
- Выявлено 20 различных видов валюты.
- Наиболее распространенная валютв USD, встречается 3224 раз.

5. employee_residence (Страна проживания сотрудника):
- Информация о странах проживания состовляет 3755 записей.
- В данных есть 78 уникальных стран.
- Наиболее частавстречающися страна проживания US, она встречается в 3004 случаях

6. company_location (Расположение главного офиса):
- Информация о главных офисах состовляет 3755 записей.
- В данных есть 72 уникальных место расположения.
- Наиболее частавстречающися место расположения офиса US, она встречается в 3040 случаях

7. company_size (Средние число сотрудников):
- Информация о сотрудниках состовляет 3755 записей.
- В данных есть 3 уникальных сотрудника.
- Наиболее наиболее частый пол сотрудника М, он встречается в 3153 случаях

In [9]:
# Проверяем на дубликаты
print("Количество дубликатов:")
print(df.duplicated().sum())

Количество дубликатов:
1171


In [10]:
# Проверка на нулевые значения
df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [ ]:
# Смотрим количество дубликатов по колонкам
for column in df.columns:
    num_distinct_values = len(df[column].unique())
    print(f"{column}: {num_distinct_values} distinct values")

work_year: 4 distinct values
experience_level: 4 distinct values
employment_type: 4 distinct values
job_title: 93 distinct values
salary: 815 distinct values
salary_currency: 20 distinct values
salary_in_usd: 1035 distinct values
employee_residence: 78 distinct values
remote_ratio: 3 distinct values
company_location: 72 distinct values
company_size: 3 distinct values


In [11]:
# Проверка на NaN значения
df.isnull().any().any()

False

In [13]:
# Выборка зарплатных признаков 
salary_features = ['salary', 'salary_currency', 'salary_in_usd']
df[salary_features]

,salary,salary_currency,salary_in_usd
0,80000,EUR,85847
1,30000,USD,30000
2,25500,USD,25500
3,175000,USD,175000
4,120000,USD,120000
...,...,...,...
3750,412000,USD,412000
3751,151000,USD,151000
3752,105000,USD,105000
3753,100000,USD,100000


##### 2. Построение модели линейной регрессии:

— Разделить данные на обучающую и тестовую выборки.

— Построить модель линейной регрессии для прогнозирования заработной платы.

— Оценить качество модели с помощью метрик MAE, MSE и R-квадрат.

— Проанализировать значимость признаков и мультиколлинеарность.

##### 3. Построение модели дерева решений:

— Построить модель дерева решений для прогнозирования заработной платы.

— Оценить качество модели с помощью метрик RMSE и R-квадрат.

— Подберите гиперпараметры для модели дерево решений

— Визуализировать дерево решений.

##### 4. Построение ансамблевой модели:

— Построить ансамблевую модель, используя Random Forest и Boosting.

— Сравнить качество ансамблевой модели с отдельными моделями.

— Оценить важность признаков в ансамблевой модели.


##### 5. Понижение размерности:

— Применить Principal Component Analysis (PCA) для снижения размерности данных.

— Построить модель линейной регрессии с использованием данных, полученных после PCA.

— Сравнить качество модели с использованием исходных данных.

##### 6. Кластеризация:

— Применить метод K-Means для кластеризации данных по заработной плате.

— Определить оптимальное количество кластеров.

— Проанализировать характеристики полученных кластеров.

— Визуализировать результаты кластеризации и проанализировать характеристики в каждом кластере.

— Проанализируйте полученные данные и сделайте выводы о том, какой алгоритм кластеризации сможет выделить кластеры лучше, чем K-Means.